In this code, I'll get the select result and make a .CSV    

In [8]:
import pandas as pd
from sqlalchemy import create_engine

make_csv:
    server = 'DESKTOP-UH0CSR8\\SQLEXPRESS'
    database = 'DB_Teste'

    connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

    engine = create_engine(connection_string)

    query = """
        SELECT 
            emp.Codigo_Empresas,
            pd.Codigo_Cliente,
            pd.Estado_Entrega_Cliente,
            ven.Codigo_Vendedores,
            pd.Pedido,
            it.produto,
            pro.Descr_Tabela,
            it.qtde,
            it.Vr_Unitario,
            it.Vr_Total,
            pd.Data_Pedido,
            pd.Data_Entrega,
            pd.Status_Pedido,
            pd.Tipo_Pedido_Desc
        FROM vw_cfc_pedido pd
        LEFT JOIN Cfc_Itens_Pedido it ON pd.pedido = it.pedido AND pd.empresa = it.empresa
        LEFT JOIN Cfc_Produtos pro ON it.Produto = pro.Codigo
        LEFT JOIN Empresas emp ON pd.Empresa = emp.Codigo_Empresas
        LEFT JOIN Rec_Vendedores ven ON pd.Nome_Vendedores = ven.Nome_Vendedores
        WHERE it.produto IS NOT NULL
        AND ven.Codigo_Vendedores IS NOT NULL
        AND pro.Descr_Tabela IS NOT NULL
        AND pd.Data_Entrega IS NOT NULL
    """

    df = pd.read_sql(query, engine)

    engine.dispose()

    df.to_csv('base.csv')
    df = pd.read_csv('base.csv')

    df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'])
    df['Data_Entrega'] = pd.to_datetime(df['Data_Entrega'])

    df['Mes_Ano_Pedido'] = df['Data_Pedido'].apply(lambda x: '{:02d}'.format(x.month)) + df['Data_Pedido'].apply(lambda x: '/{:04d}'.format(x.year))
    df['Mes_Ano_Entrega'] = df['Data_Entrega'].apply(lambda x: '{:02d}'.format(x.month)) + df['Data_Entrega'].apply(lambda x: '/{:04d}'.format(x.year))
